# Create Labels for the Audio Sequences
The purpose of this notebook is to generate the target labels for supervised training. This is somewhat manual and the following procedure was used:
 - open an audio file in [Audacity](http://www.audacityteam.org)
 - by visual inspection of the waveform locate the start and end samples of the target sequence (verify selection by playing the audio in that range)
 - using a window of 3.45 seconds (the width of the target sequence in this case) label the time-correlated mel spectrograph observation as class 1, otherwise class 0.

The audio file contents are as follows:

** Washing Machine **
``` 
File :161225-000
Beep Sequence Count: 0

File :161225-001
Beep Sequence Count: 1
Start Sample: 9541250
End Sample:  9596500

File :161225-002
Beep Sequence Count: 1
Start Sample: 713600
End Sample:  721360

File :161225-003
Beep Sequence Count: 1
Start Sample: 813800
End Sample:  869300

File :161225-004
Beep Sequence Count: 1
Start Sample: 808700
End Sample:  864000

File :161225-005
Beep Sequence Count: 1
Start Sample: 686400
End Sample:  742000
```
** Dryer **
```
File :161225-006
Beep Sequence Count: 3
Start Sample: 201850
End Sample: 262330
Start Sample: 1156480
End Sample:  1212300
Start Sample: 2110750
End Sample:  2118300
```

Based on this data we will create the label vector for each mel spectrogram row. 

**Note:** we need to take into account the sliding 4 second windows with a stride of 10ms.

In [1]:
%matplotlib inline
import seaborn
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os

source_directory = r'/Volumes/ThorsHammer/Data Science/data/audio-recognition/16k/'
mel_directory = r'/Volumes/ThorsHammer/Data Science/data/audio-recognition/mel/'
stride = 0.01
sample_length = 4 #seconds
sample_rate = 16000

# This following represent the first postive class observation in the data
positive_classes = {
    '161225-001' : 9541250, # Large file takes too long for me to wait.
    '161225-002' : 713600,
    '161225-003' : 813800,
    '161225-004' : 808700,
    '161225-005' : 686400
}

file_sufix = '.wav-mel.csv'

### Helper functions

In [2]:
#Determine the observation index for a given sample numer
def calc_mel_observation_index(raw_sample_number):
    observation_index = int(raw_sample_number / (sample_rate * stride))
    return observation_index

def file_len(filename):
    with open(filename) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def list_all_files(source_directory, truncate_extension=False, extension='.wav'):
    # Find the audio wav files in the source folder
    raw_files = []
    for file in os.listdir(source_directory):
        if file.endswith(extension):
            if(truncate_extension):
                file = file[:file.rfind('.')]
            raw_files.append(file)
            #print(raw_files[-1])
            
    return raw_files

def save_labels(destination_directory, prefix, labels):
    output_file = '{0}{1}-mel-labels.csv'.format(destination_directory,prefix)
    np.savetxt(output_file,labels,delimiter=',', fmt='%1i')
    print('Done: %s'%output_file)

### Loop through the audio files and generate the corresponding label vector
The labels are written to disk.

In [5]:
%%time

all_files = list_all_files(source_directory, truncate_extension=True)
file_len_dict = {}
file_label_dict = {}

for file in sorted(all_files):
    
    filename = os.path.join(mel_directory,file+file_sufix)
    num_observations = file_len(filename)
    file_len_dict[file] = num_observations

    labels = np.zeros(num_observations)
    
    if(file in positive_classes.keys()):
        print('%s contains target'%file)
        start_sample = positive_classes[file]
        target_class_range_start = calc_mel_observation_index(start_sample - (sample_rate * 0.5))
        target_class_range_end = calc_mel_observation_index(start_sample)
        labels[target_class_range_start:target_class_range_end] = 1
        file_label_dict[file] = labels
    else:
        print('%s target absent'%file)
    save_labels(mel_directory, file, labels)

print (file_len_dict)

161225-000 target absent
Done: /Volumes/ThorsHammer/Data Science/data/audio-recognition/mel/161225-000-mel-labels.csv
161225-001 contains target
Done: /Volumes/ThorsHammer/Data Science/data/audio-recognition/mel/161225-001-mel-labels.csv
161225-002 contains target
Done: /Volumes/ThorsHammer/Data Science/data/audio-recognition/mel/161225-002-mel-labels.csv
161225-003 contains target
Done: /Volumes/ThorsHammer/Data Science/data/audio-recognition/mel/161225-003-mel-labels.csv
161225-004 contains target
Done: /Volumes/ThorsHammer/Data Science/data/audio-recognition/mel/161225-004-mel-labels.csv
161225-005 contains target
Done: /Volumes/ThorsHammer/Data Science/data/audio-recognition/mel/161225-005-mel-labels.csv
161225-006 target absent
Done: /Volumes/ThorsHammer/Data Science/data/audio-recognition/mel/161225-006-mel-labels.csv
{'161225-006': 15383, '161225-004': 5419, '161225-005': 4684, '161225-002': 4742, '161225-003': 5456, '161225-000': 39038, '161225-001': 60532}
CPU times: user 7.03